In [1]:
import numpy as np

# Kalman Filter Algorithm

**Initialization**

    1. Initialize the state of the filter
    2. Initialize our belief in the state
    
**Predict**

    1. Use process model to predict state at the next time step
    2. Adjust belief to account for the uncertainty in prediction    
**Update**

    1. Get a measurement and associated belief about its accuracy
    2. Compute residual between estimated state and measurement
    3. Compute scaling factor based on whether the measurement
    or prediction is more accurate
    4. set state between the prediction and measurement based 
    on scaling factor
    5. update belief in the

# Equation Reference

$$
\begin{aligned}
\text{Predict Step}\\
\mathbf{\bar x} &= \mathbf{F x} + \mathbf{B u} = PriorState\\ 
\mathbf{\bar P} &= \mathbf{FP{F}}^\mathsf T + \mathbf Q = ProcessNoise\\
\\
\text{Update Step}\\
\textbf{S} &= \mathbf{H\bar PH}^\mathsf T + \mathbf R = ObtainSystemUncertainty\\
\mathbf K &= \mathbf{\bar PH}^\mathsf T \mathbf{S}^{-1} = KalmanGain\\
\textbf{y} &= \mathbf z - \mathbf{H \bar x} = Residual\\
\mathbf x &=\mathbf{\bar x} +\mathbf{K\textbf{y}} = UpdatedState(Posterior)\\
\mathbf P &= (\mathbf{I}-\mathbf{KH})\mathbf{\bar P = UpdateNoise}
\end{aligned}
$$

#### Initial State (x)

$$\mathbf x 
= \begin{bmatrix}position (m)\\ velocity (m/s)\end{bmatrix} 
= \begin{bmatrix}x\\ v\end{bmatrix} 
= \begin{bmatrix}0\\ 0\end{bmatrix}$$

#### State transition function (F)

$$\mathbf F = 
\begin{bmatrix}1&\Delta t  \\ 0&1\end{bmatrix}$$

#### Control Variable (u)

$$\mathbf u = 
\begin{bmatrix}a\end{bmatrix}$$

#### Control Matrix (B)

$$\mathbf B = 
\begin{bmatrix}1/2\Delta t^2 \\ \Delta t\end{bmatrix}$$

#### Covariance Matrix (P)

Position $\mathbf x$ variance ($\sigma_\mathtt{x}^2$) selected at 5m 

Velocity $\mathbf v$ variance ($\sigma_\mathtt{v}^2$) selected at 5m/s

Covariance $\sigma_\mathtt{xv} = \sigma_\mathtt{x}\sigma_\mathtt{v} $

$$\mathbf P 
= \begin{bmatrix}\sigma_\mathtt{x}^2&\sigma_x\sigma_v  \\ \sigma_v\sigma_x&\sigma_\mathtt{v}^2\end{bmatrix} 
= \begin{bmatrix}5&5  \\ 5&5\end{bmatrix}$$

#### Process Noise (Q)

$$\mathbf Q 
= \begin{bmatrix}0&0 \\ 0&1\end{bmatrix}
$$

#### Measurement Noise (R)

Need to double check sensors

$$\mathbf x 
= \begin{bmatrix}100\end{bmatrix}
$$

### State/Measurement Related Initialization

In [2]:
state                = np.array([[0],                  # state
                                 [0]])

print(state)

time_step            = 1                               # time step

state_transition     = np.array([[1, time_step],       # state transition matrix
                                 [0, 1]])

control_input        = np.array([10])                  # control input 

control_matrix       = np.array([[0.5*(time_step**2)], # control matrix
                                 [time_step]])

measurement_data     = np.array([10])                  # measurement
  
measurement_function = np.array([1,0])                 # measurement function
                                

[[0]
 [0]]


### Noise Related Initialization

In [3]:
position_variance = 5
velocity_variance = 5
position_std      = np.sqrt(position_variance)
velocity_std      = np.sqrt(velocity_variance)
covariance_value  = position_std * velocity_std

covariance_matrix = np.array([[position_variance, covariance_value],
                              [covariance_value,  position_variance]])

process_noise     = np.array([[0,0],
                              [0,1]]) 

measurement_noise = np.array([4])

# Math in terms of Matrices

$$
\begin{aligned}
\text{Prediction Step}\\
\mathbf{\bar x} &= \mathbf{F x} + \mathbf{B u} = PriorState\\ 
\mathbf{\bar P} &= \mathbf{FP{F}}^\mathsf T + \mathbf Q = ProcessNoise\\
\end{aligned}
$$


$$
\begin{aligned}
\mathbf{\bar x} &= \mathbf{F x} + \mathbf{B u}\\ 
\mathbf{\bar x} &= \begin{bmatrix}1&\Delta t  \\ 0&1\end{bmatrix} \begin{bmatrix}x  \\ v\end{bmatrix} 
+ \begin{bmatrix}1/2\Delta t^2 \\ \Delta t\end{bmatrix} 
\begin{bmatrix}a\end{bmatrix} \\
\mathbf{\bar x} &= \begin{bmatrix}1&1  \\ 0&1\end{bmatrix} \begin{bmatrix}0  \\ 0\end{bmatrix} + 
\begin{bmatrix}0.5*1 \\ 1\end{bmatrix} 
\begin{bmatrix}10\end{bmatrix} \\
\mathbf{\bar x} &= \begin{bmatrix}5 \\ 10\end{bmatrix} 
\end{aligned}
$$

$$
\begin{aligned}
\mathbf{\bar P} &= \mathbf{FP{F}}^\mathsf T + \mathbf Q = ProcessNoise\\
\mathbf{\bar x} &= \begin{bmatrix}1&\Delta t  \\ 0&1\end{bmatrix} \begin{bmatrix}\sigma_\mathtt{x}^2&\sigma_x\sigma_v \\ \sigma_v\sigma_x&\sigma_\mathtt{v}^2\end{bmatrix} \begin{bmatrix}1&0  \\ \Delta t&1\end{bmatrix}
+
\begin{bmatrix}0&0  \\ 0&1\end{bmatrix}
\end{aligned}
$$

In [4]:
# Calculate Prior State
Fx = np.dot(state_transition, state)                   # Calculate for Fx
Bu = np.array([np.dot(control_matrix, control_input)]) # Calculate for Bu
state = np.add(Fx, Bu.T) # Add Fx and Bu, Bu is transposed because it needs same dimensions as Fx


covariance_matrix = np.dot(state_transition,covariance_matrix) # Calculate process noise
covariance_matrix = np.dot(covariance_matrix,state_transition.T)
covariance_matrix = covariance_matrix + process_noise

print("State is")
print(state)
print("\nCovariance is")
print(covariance_matrix)

State is
[[ 5.]
 [10.]]

Covariance is
[[20. 10.]
 [10.  6.]]


$$
\begin{aligned}
\text{Update Step}\\
\textbf{S} &= \mathbf{H\bar PH}^\mathsf T + \mathbf R = ObtainSystemUncertainty\\
\mathbf K &= \mathbf{\bar PH}^\mathsf T \mathbf{S}^{-1} = KalmanGain\\
\textbf{y} &= \mathbf z - \mathbf{H \bar x} = Residual\\
\mathbf x &=\mathbf{\bar x} +\mathbf{K\textbf{y}} = UpdatedState(Posterior)\\
\mathbf P &= (\mathbf{I}-\mathbf{KH})\mathbf{\bar P = UpdateNoise}
\end{aligned}
$$

In [5]:
# Obtain system uncertainty for sensor
HP = np.dot(covariance_matrix,measurement_function) # Calculate for HP
HPH = np.dot(covariance_matrix.T,HP)                # Calculate for HP(H^T)
measurement_uncertainty = np.add(HPH,measurement_noise) # Calculate for S = HP(H^T) + R
print("Measurement Uncertainty is")
print(measurement_uncertainty)

# Obtain Kalman Gain
PH = np.dot(covariance_matrix,measurement_function) # Calculate for PH
K = np.divide(PH,measurement_uncertainty) # Calculate for K = PH/S
kalman_gain = np.array([[K[0]],[K[1]]])
print("\nKalman Gain is")
print(kalman_gain)

# Obtain Residual
Hx = np.dot(measurement_function,state) # Calculate for Hx
residual = np.subtract(measurement_data,Hx) # Calculate for y = z - Hx
print("\nResidual is")
print(residual)

# Update State
Ky = np.dot(kalman_gain,residual)
state = np.add(state,[[Ky[0]],[Ky[1]]])
print("\nUpdated State is")
print(state)

# Update Covariance 
KH = np.dot(measurement_function,kalman_gain)
KH_minus = np.subtract(np.identity(2),KH)
covariance_matrix = np.dot(KH_minus,covariance_matrix)
print("\nUpdated Covariance is")
print(covariance_matrix)

Measurement Uncertainty is
[504. 264.]

Kalman Gain is
[[0.03968254]
 [0.03787879]]

Residual is
[5.]

Updated State is
[[ 5.1984127 ]
 [10.18939394]]

Updated Covariance is
[[18.80952381  9.36507937]
 [ 8.80952381  5.36507937]]


##